In [ ]:
!pip install transformers

In [ ]:
!pip install requests beautifulsoup4 pandas numpy

## Pipeline

In [ ]:
from transformers import pipeline

### 1. Sentiment Analysis

In [ ]:
classifier = pipeline('sentiment-analysis')

In [ ]:
classifier("I hate that movie and the cast was bad ")

In [ ]:
classifier2 = pipeline('sentiment-analysis', model='bert-base-uncased')

In [ ]:
classifier2("I loved that movie and the cast was good ")

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it."])
for i in results:
    print(f"{i['label']}, {round(i['score'],2)}")

### 2. Generator

In [ ]:
generator = pipeline('text-generation')

In [ ]:
print(generator("As far as I am concerned, I will",))

### 3. Fill Mask

In [ ]:
unmasker = pipeline("fill-mask")

In [ ]:
from pprint import pprint
pprint(unmasker(f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."))

### 4. NER

In [ ]:
ner_pipe = pipeline("ner")

In [ ]:
sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

In [ ]:
print(ner_pipe(sequence))

### 5. Question Answering

In [ ]:
question_answerer = pipeline("question-answering")


In [ ]:
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

In [ ]:
result = question_answerer(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

### 6. Summarization

In [ ]:
summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [ ]:
print(summarizer(ARTICLE))

### 7. Translation

In [ ]:
translator = pipeline("translation_en_to_de")

In [ ]:
print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

# 1.Do something on Real Data (Sentiment Analysis on Yelp - Social Brew Cafe)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

## Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

In [ ]:
int(torch.argmax(result.logits))+1

## Collect Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

## Load Reviews into DataFrame and Score

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

In [ ]:
df['review'].iloc[3]

# 2.Do something on Real Data (Translator + Gradio)

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr   

## Load Up Pipeline

In [ ]:
translation_pipeline = pipeline('translation_en_to_de')

In [ ]:
results = translation_pipeline('I love Juventus')

In [ ]:
results[0]['translation_text']

## Create Gradio Function and Interface

In [ ]:
def translate_transformers(from_text):
    results = translation_pipeline(from_text)
    return results[0]['translation_text']

In [ ]:
translate_transformers('My name is Soheil')

In [ ]:
interface = gr.Interface(fn=translate_transformers, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Text to translate'),
                        outputs='text')

In [ ]:
interface.launch(share=True)

# 3.Do something on Real Data (Summarization Medium)

In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [ ]:
summarizer = pipeline("summarization")

## Get Blog Post from Medium

In [ ]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"

In [ ]:
r = requests.get(URL)

In [ ]:
r

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
ARTICLE

## Chunk Text

In [ ]:
max_chunk = 500

In [ ]:
ARTICLE = ARTICLE.replace('.', '')
ARTICLE = ARTICLE.replace('?', '')
ARTICLE = ARTICLE.replace('!', '')

In [ ]:
sentences = ARTICLE.split(' ')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [ ]:
len(chunks)

In [ ]:
chunks[0]

## Summarize Text

In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
res[0]

# Your Turn

Summarize this article:

URL-test = https://medium.com/analytics-vidhya/openai-gpt-3-language-models-are-few-shot-learners-82531b3d3122


# Persian

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

# v3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# model = TFAutoModel.from_pretrained(model_name_or_path)  For TF
model = AutoModel.from_pretrained(model_name_or_path)

text = "ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenizer.tokenize(text)

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
config = AutoConfig.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
config = AutoConfig.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")
model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-snappfood")

In [ ]:
model.save_pretrained("/content/HooshvareLab-bert-fa-base-uncased-sentiment-snappfood")
tokenizer.save_pretrained("/content/HooshvareLab-bert-fa-base-uncased-sentiment-snappfood")

In [ ]:
nlp_sentence_classif = pipeline('sentiment-analysis', model="/content/HooshvareLab-bert-fa-base-uncased-sentiment-snappfood/")
nlp_sentence_classif('این خوراک به شدت خوشمزه است')